In [842]:
## Assignment
import gzip
import json
import pandas as pd
import re
import mysql.connector
from sqlalchemy import create_engine
from mysql.connector import Error
import enum
from sqlalchemy import MetaData, Table, Column, Integer, String, Boolean, Text, BigInteger, Float, TIMESTAMP, Enum


## 1. Review Existing Unstructured Data and Diagram a New Structured Relational Data Model


### 1a. Read json data into DataFrame Objects

In [843]:
jsonFiles = ["brands.json.gz", "receipts.json.gz", "users.json.gz"]

def readJsonGz(file_path):
    """
    Given a path to a json.gz file, `file_path`
    Read the data into a normalized pandas DataFrame object

    Args:
        file_path(str): The path to a gzip file

    Returns:
        a pandas DataFrame object containing data in `file_path`
    """

    with gzip.open(file_path, 'r') as data_file:    
        # data = json.load(data_file)  
        data = data_file.read()

    data = data.decode("utf-8")
    lines = data.split('\n')

    for i, line in enumerate(lines):
        if line.strip():  # Skip empty lines
            try:
                json.loads(line)
            except json.JSONDecodeError as e:
                print(f"Invalid JSON on line {i+1}: {e}")
                print(f"Content: {line}")
                break

    data = "[" + ",".join(line for line in lines if line.strip()) + "]"
    data = json.loads(data)
    normalized_data = pd.json_normalize(data)
    return normalized_data

def extractData(line):
    """
    Use regex to extract data wrapped between '{' and '}' in an input string.
    Args: 
        line(str): The data stored in a line of json file

    Returns: 
        substring of `line`, contains characters wrapped around by {}
    """
    pattern = r'\{.*\}'

    # Search for the pattern in the input string
    match = re.search(pattern, line)
    json_object =""

    if match:
        json_object = match.group(0)
    return json_object


def read_json_gz_skip_gzip_lines(file_path):
    """
    Given a path to a json.gz file, `file_path`
    Read the data into a json object and return

    Args:
        file_path(str): The path to a gzip file
        
    Returns:
        a json object containing data in `file_path`
    """
    valid_lines = []

    # Open the gzip file
    with gzip.open(file_path, 'r') as f:
        d = f.read()
        lines = d.decode("utf-8").split("\n")
        valid_lines = []
        for line in lines:
            extracted = extractData(line)
            if extracted != "":
                valid_lines.append(extracted)
    json_string =  "[" + ",".join(valid_lines) + "]"
    data = json.loads(json_string)
    
    return data

rewardsReceiptItemsLst = []
num_nan = 0


def normalize_rewardsReceiptItems(row, dest = rewardsReceiptItemsLst):
    """
    Extract 'reward receipt item' from column `rewardsReceiptItemList`
    and map receipt id to each item.

    Args:
        row(Pandas series): a pandas series sliced from a dataframe as a row
        dest(list): a list where each processed item will be appended to
    
    Returns:
        None
    """
    global num_nan
    receipt_id = row["_id"]
    rewardsReceiptItemList = row["rewardsReceiptItemList"]
    if pd.isna(rewardsReceiptItemList).all() if isinstance(rewardsReceiptItemList, (list, pd.Series)) else pd.isna(rewardsReceiptItemList):
        num_nan += 1
        return 
    new_row = pd.json_normalize(rewardsReceiptItemList)
    new_row["receiptId"] = receipt_id
    dest.append(new_row)



Found that users.json.gz is not able to be imported due to unexpected symbols.  
Unzip it, and try to investigate

In [844]:
dataDict = {}
for file in jsonFiles:
    try:
        data= readJsonGz(file)
        dataDict[file.split(".")[0]] = data
    except json.JSONDecodeError as e:
        print(e)
        print("Fail to read {}".format(file))


Invalid JSON on line 1: Expecting value: line 1 column 1 (char 0)
Content: users.json                                                                                          0100777 0000000 0000000 00000254301 14567170355 010072  0                                                                                                    ustar 00                                                                                                                                                                                                                                                       {"_id":{"$oid":"5ff1e194b6a9d73a3a9f1052"},"active":true,"createdDate":{"$date":1609687444800},"lastLogin":{"$date":1609687537858},"role":"consumer","signUpSource":"Email","state":"WI"}
Expecting value: line 1 column 2 (char 1)
Fail to read users.json.gz


In [845]:
# !gzip -dk users.json.gz

Opened the users.json file and found that  
1)  in the first line,
there're bunch of "\x00" characters in front of a actual data row.  
2) In the last row, it's full of "\x00".  
Therefore, created a function `read_json_gz_skip_gzip_lines` to extract data placed between "{}" in each line

In [846]:
file = jsonFiles[2]
try:
    data= read_json_gz_skip_gzip_lines(file)
    dataDict[file.split(".")[0]] = pd.json_normalize(data)
except json.JSONDecodeError:
    print(e)
    print(file)
dataDict.keys()

dict_keys(['brands', 'receipts', 'users'])

In [847]:
dataDict.keys()

dict_keys(['brands', 'receipts', 'users'])

In [848]:
def cleanColName(col):
    splitted = col.split(".")
    if len(splitted) <= 2:
        return splitted[0]
    else:
        return col.replace("$", "").replace(".", "_")

In [849]:
brandsData = dataDict["brands"]
receiptsData = dataDict["receipts"]
usersData = dataDict["users"]


usersData.role = usersData.role.apply(str.upper)
usersDataRenameCols = {col: cleanColName(col) for col in usersData.columns if "." in col}
usersData = usersData.rename(columns=usersDataRenameCols)

brandsDataRenameCols = {'_id.$oid': '_id',
                        'cpg.$id.$oid': 'cpg_id_oid', 'cpg.$ref': 'cpg_ref'}
brandsData = brandsData.rename(columns=brandsDataRenameCols)

receiptsDataRenameCols = {col: cleanColName(col) for col in receiptsData.columns if "." in col}
receiptsData = receiptsData.rename(columns=receiptsDataRenameCols)



In [850]:
for col in ['createdDate','lastLogin']:
    usersData[col] = pd.to_datetime(usersData[col], unit="ms")

In [851]:
for col in ['createDate', 'dateScanned',
        'finishedDate', 'modifyDate', 
        'pointsAwardedDate', 'purchaseDate']:
    receiptsData[col] = pd.to_datetime(receiptsData[col], unit="ms")

Found that Receipt dataframe has a column that contains list of other objects: *rewardsReceiptItemList*

In [852]:
receiptsData.head()

,bonusPointsEarned,bonusPointsEarnedReason,pointsEarned,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId,_id,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,purchaseDate
0,500.0,"Receipt number 2 completed, bonus point schedu...",500.0,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6,5ff1e1eb0a720f0523000575,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,2021-01-03 00:00:00
1,150.0,"Receipt number 5 completed, bonus point schedu...",150.0,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052,5ff1e1bb0a720f052300056b,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,2021-01-02 15:24:43
2,5.0,All-receipts receipt bonus,5,1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b,5ff1e1f10a720f052300057a,2021-01-03 15:25:37,2021-01-03 15:25:37,NaT,2021-01-03 15:25:42,NaT,2021-01-03 00:00:00
3,5.0,All-receipts receipt bonus,5.0,4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6,5ff1e1ee0a7214ada100056f,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:39,2021-01-03 15:25:34,2021-01-03 00:00:00
4,5.0,All-receipts receipt bonus,5.0,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.00,5ff1e194b6a9d73a3a9f1052,5ff1e1d20a7214ada1000561,2021-01-03 15:25:06,2021-01-03 15:25:06,2021-01-03 15:25:11,2021-01-03 15:25:11,2021-01-03 15:25:06,2021-01-02 15:25:06


In [853]:
# Rereceipts table has a column that contains list of other objects
receiptsData.apply(normalize_rewardsReceiptItems, axis = 1)
rewardsReceiptItems = pd.concat(rewardsReceiptItemsLst, ignore_index= True)
rewardsReceiptItems.head()

,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,userFlaggedNewItem,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
0,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,True,1.0,028400642255,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,False,1,True,NaN,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4.0,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Validate data integrity after applying data extraction to receipt table

In [854]:
actual_nan_rri = receiptsData["_id"].count() - receiptsData.rewardsReceiptItemList.count()

report1 = """
During extracting receipt reward items, found {} receipts that nan value in 'receiptsRewordsItemList'\n
Matched with actual missing values in the column: {}\n
Total number of receipt rewards items extracted: {}\n
"""
print(report1.format(num_nan, actual_nan_rri == num_nan, rewardsReceiptItems.shape[0] ))



During extracting receipt reward items, found 440 receipts that nan value in 'receiptsRewordsItemList'

Matched with actual missing values in the column: True

Total number of receipt rewards items extracted: 6941




In [855]:
receiptsData.columns

Index(['bonusPointsEarned', 'bonusPointsEarnedReason', 'pointsEarned',
       'purchasedItemCount', 'rewardsReceiptItemList', 'rewardsReceiptStatus',
       'totalSpent', 'userId', '_id', 'createDate', 'dateScanned',
       'finishedDate', 'modifyDate', 'pointsAwardedDate', 'purchaseDate'],
      dtype='object')

In [856]:
receiptsCols = ['bonusPointsEarned', 'bonusPointsEarnedReason', 'pointsEarned',
                'purchasedItemCount', 'rewardsReceiptStatus',
                'totalSpent', 'userId', '_id', 'createDate',
                'dateScanned', 'finishedDate', 'modifyDate',
                'pointsAwardedDate', 'purchaseDate']
receiptsData = receiptsData[receiptsCols]
receiptsData

,bonusPointsEarned,bonusPointsEarnedReason,pointsEarned,purchasedItemCount,rewardsReceiptStatus,totalSpent,userId,_id,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,purchaseDate
0,500.0,"Receipt number 2 completed, bonus point schedu...",500.0,5.0,FINISHED,26.00,5ff1e1eacfcf6c399c274ae6,5ff1e1eb0a720f0523000575,2021-01-03 15:25:31.000,2021-01-03 15:25:31.000,2021-01-03 15:25:31,2021-01-03 15:25:36.000,2021-01-03 15:25:31,2021-01-03 00:00:00
1,150.0,"Receipt number 5 completed, bonus point schedu...",150.0,2.0,FINISHED,11.00,5ff1e194b6a9d73a3a9f1052,5ff1e1bb0a720f052300056b,2021-01-03 15:24:43.000,2021-01-03 15:24:43.000,2021-01-03 15:24:43,2021-01-03 15:24:48.000,2021-01-03 15:24:43,2021-01-02 15:24:43
2,5.0,All-receipts receipt bonus,5,1.0,REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b,5ff1e1f10a720f052300057a,2021-01-03 15:25:37.000,2021-01-03 15:25:37.000,NaT,2021-01-03 15:25:42.000,NaT,2021-01-03 00:00:00
3,5.0,All-receipts receipt bonus,5.0,4.0,FINISHED,28.00,5ff1e1eacfcf6c399c274ae6,5ff1e1ee0a7214ada100056f,2021-01-03 15:25:34.000,2021-01-03 15:25:34.000,2021-01-03 15:25:34,2021-01-03 15:25:39.000,2021-01-03 15:25:34,2021-01-03 00:00:00
4,5.0,All-receipts receipt bonus,5.0,2.0,FINISHED,1.00,5ff1e194b6a9d73a3a9f1052,5ff1e1d20a7214ada1000561,2021-01-03 15:25:06.000,2021-01-03 15:25:06.000,2021-01-03 15:25:11,2021-01-03 15:25:11.000,2021-01-03 15:25:06,2021-01-02 15:25:06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114,25.0,COMPLETE_NONPARTNER_RECEIPT,25.0,2.0,REJECTED,34.96,5fc961c3b8cfca11a077dd33,603cc0630a720fde100003e6,2021-03-01 10:22:27.000,2021-03-01 10:22:27.000,NaT,2021-03-01 10:22:28.000,NaT,2020-08-17 00:00:00
1115,NaN,NaN,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33,603d0b710a720fde1000042a,2021-03-01 15:42:41.873,2021-03-01 15:42:41.873,NaT,2021-03-01 15:42:41.873,NaT,NaT
1116,NaN,NaN,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33,603cf5290a720fde10000413,2021-03-01 14:07:37.664,2021-03-01 14:07:37.664,NaT,2021-03-01 14:07:37.664,NaT,NaT
1117,25.0,COMPLETE_NONPARTNER_RECEIPT,25.0,2.0,REJECTED,34.96,5fc961c3b8cfca11a077dd33,603ce7100a7217c72c000405,2021-03-01 13:07:28.000,2021-03-01 13:07:28.000,NaT,2021-03-01 13:07:29.000,NaT,2020-08-17 00:00:00


In [857]:
rewardsReceiptItems.metabriteCampaignId.value_counts().index

Index(['BEN AND JERRYS ICE CREAM',
       'KLEENEX TRUSTED CARE FACIAL TISSUES 120 - 179 COUNT, 8 PACK',
       'PEPSI 12 OZ 12 PACK', 'DORITOS NACHO CHEESE MULTI SERVE',
       'KNORR SIDES', 'KRAFT MACARONI & CHEESE DINNER - BOX',
       'HIDDEN VALLEY SALAD DRESSING 21OZ AND LARGER',
       'JUST BARE FRESH CHICKEN BREAST FILETS',
       'MARIE CALLENDER'S POT PIE MULTI SERVE', 'KRAFT SINGLES',
       'DOLE 100% FRUIT JUICES', 'DOLE BLENDS MULTI SERVE',
       'LIPTON RECIPE SECRETS', 'BEST FOODS MAYONNAISE',
       'GM CEREAL VARIETY PACKS', 'YUBAN COFFEE',
       'JELL-O REFRIGERATED GELATIN & PUDDING', 'CLASSICO SAUCE',
       'CINNAMON TOAST CRUNCH CEREAL GIANT SIZE', 'CHEX CEREAL MEDIUM SIZE',
       'WISHBONE SALAD DRESSING', 'HONEY NUT CHEERIOS CEREAL MEDIUM SIZE',
       'VELVEETA MACARONI & CHEESE DINNER', 'PLANTERS CASHEWS',
       'KNORR RECIPE MIXES', 'QUAKER CEREAL & OATS', 'PEPSI 12 OZ 24+ COUNT',
       'KRAFT DRESSING - INDULGENCE',
       'HUGGIES NATURAL CARE SENSI

In [858]:
receiptsData.rewardsReceiptStatus.value_counts()

rewardsReceiptStatus
FINISHED     518
SUBMITTED    434
REJECTED      71
PENDING       50
FLAGGED       46
Name: count, dtype: int64

In [917]:

def create_connection(host_name, user_name, user_password, db_name=None):
    """
    Creates a connection to the MySQL database.

    Args:
        host_name (str): The hostname of the MySQL server.
        user_name (str): The username to authenticate with.
        user_password (str): The password to authenticate with.
        db_name (str, optional): The name of the database to connect to. Defaults to None.

    Returns:
        mysql.connector.connection_cext.CMySQLConnection: The connection object to the MySQL database.
    """

    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("Connection to MySQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

def create_sqlalchemy_engine(user, password, host, db):
    """
    Creates a SQLAlchemy engine for connecting to the MySQL database.

    Args:
        user (str): The username to authenticate with.
        password (str): The password to authenticate with.
        host (str): The hostname of the MySQL server.
        db (str): The name of the database to connect to.

    Returns:
        sqlalchemy.engine.Engine: The SQLAlchemy engine connected to the MySQL database.
    """

    url = f'mysql+mysqlconnector://{user}:{password}@{host}/{db}'
    engine = create_engine(url)
    return engine


def execute_query(conn, query):
    """
    Executes a SQL query on the MySQL database.

    Args:
        connection (mysql.connector.connection_cext.CMySQLConnection): The connection object to the MySQL database.
        query (str): The SQL query to execute.

    Returns:
        None
    """

    cursor = conn.cursor()
    try:
        cursor.execute(query)
        conn.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")


def query_to_df(conn, query):
    """
    Executes a SQL SELECT query and returns the result as a pandas DataFrame.

    Args:
        connection (mysql.connector.connection_cext.CMySQLConnection): The connection object to the MySQL database.
        query (str): The SQL SELECT query to execute.

    Returns:
        pandas.DataFrame: A DataFrame containing the query results.
    """
    
    try:
        df = pd.read_sql(query, conn)
        return df
    except Error as e:
        print(f"The error '{e}' occurred")
        return None

db = 'fetch_db'
conn = create_connection("localhost", "root", "new_password", db)
engine = create_sqlalchemy_engine('root', 'new_password', 'localhost', db)


Connection to MySQL DB successful


Create database in the MySQL server

In [918]:
# create_db_query = f"""CREATE DATABASE {db}"""
use_db_query = f"""USE {db}"""

# execute_query(conn, create_db_query)
execute_query(conn, use_db_query)

Query executed successfully


Load tables into Database

a) Load user table into Database

In [879]:
class RewardsStatus(enum.Enum):
    FINISHED = "FINISHED"
    SUBMITTED = "SUBMITTED"
    REJECTED = "REJECTED"
    PENDING = "PENDING"
    FLAGGED = "FLAGGED"

class UserRole(enum.Enum):
    CONSUMER = "CONSUMER"
    FETCH_STAFF = "FETCH_STAFF"

USER_TABLE = "User"
RECEIPT_TABLE = "Receipt"
BRAND_TABLE = "Brand"
ITEM_TABLE = "ReceiptRewardItem"



metadata = MetaData()

# Define the table schema
user_table = Table(
    USER_TABLE, metadata,
    Column('active', Boolean),
    Column('role', Enum(UserRole, name="UserRole")),
    Column('signUpSource', Text),
    Column('state', Text),
    Column('_id',  String(255), primary_key=True),
    Column('createdDate', TIMESTAMP),
    Column('lastLogin', TIMESTAMP)
)


# Define the table schema
receipt_table = Table(
    RECEIPT_TABLE, metadata,
    Column('_id', String(255), primary_key=True),
    Column('bonusPointsEarned', Integer),
    Column('bonusPointsEarnedReason', Text),
    Column('pointsEarned', Integer),
    Column('purchasedItemCount', Integer),
    Column('totalSpent', Float),
    Column('userId', String(255)),
    Column('rewardsReceiptStatus', Enum(RewardsStatus, name="RewardsStatus")),
    Column('createDate', TIMESTAMP),
    Column('dateScanned', TIMESTAMP),
    Column('finishedDate', TIMESTAMP),
    Column('modifyDate', TIMESTAMP),
    Column('pointsAwardedDate', TIMESTAMP),
    Column('purchaseDate', TIMESTAMP)
)

brand_table = Table(
    BRAND_TABLE, metadata,
    Column('_id',  String(255), primary_key=True),
    Column('barcode', String(255)),
    Column('category', String(255)),
    Column('categoryCode', String(255)),
    Column('name', String(255)),
    Column('topBrand', Boolean),
    Column('cpg_id_oid', String(255)),
    Column('cpg_ref', String(255)),
    Column('brandCode', String(255))
)

receipt_reward_item = Table(
    ITEM_TABLE, metadata,
    Column('barcode', String(255)),
    Column('brandCode', String(255)),
    Column('receiptId', String(255)),
    Column('partnerItemId', String(255)),
    Column('pointsPayerId', String(255)),
    Column('rewardsProductPartnerId', String(255)),
    Column('metabriteCampaignId', String(255)),
    Column('description', String(255)),
    Column('finalPrice', Float),
    Column('itemPrice', Float),
    Column('needsFetchReview', Boolean),
    Column('preventTargetGapPoints', Boolean),
    Column('quantityPurchased', Integer),
    Column('userFlaggedBarcode', String(255)),
    Column('userFlaggedNewItem', Boolean),
    Column('userFlaggedPrice', Float),
    Column('userFlaggedQuantity', Integer),
    Column('needsFetchReviewReason', String(255)),
    Column('pointsNotAwardedReason', String(255)),
    Column('rewardsGroup', String(255)),
    Column('userFlaggedDescription', String(255)),
    Column('originalMetaBriteBarcode', String(255)),
    Column('originalMetaBriteDescription', String(255)),
    Column('competitorRewardsGroup', String(255)),
    Column('discountedItemPrice', Float),
    Column('originalReceiptItemText', String(255)),
    Column('itemNumber', String(255)),
    Column('originalMetaBriteQuantityPurchased', Integer),
    Column('pointsEarned', Float),
    Column('targetPrice', Float),
    Column('competitiveProduct', Boolean),
    Column('originalFinalPrice', Float),
    Column('originalMetaBriteItemPrice', Float),
    Column('deleted', Boolean),
    Column('priceAfterCoupon', Float)
)

metadata.create_all(engine)

In [880]:
# Add foreign key constraints, during this process, 
# I found that not all use ids referenced in Receipt table exist in the User table
# set_constraint_receipt_query = f"""
#         ALTER TABLE {RECEIPT_TABLE}
#         ADD FOREIGN KEY (userId) REFERENCES User(_id);"""

set_constraint_item_query = f"""
        ALTER TABLE {ITEM_TABLE}
        ADD FOREIGN KEY (receiptId) REFERENCES Receipt(_id);"""

In [881]:
# execute_query(conn, set_constraint_receipt_query)
execute_query(conn, set_constraint_item_query)


Query executed successfully


In [864]:
usersData.role.value_counts()

role
CONSUMER       413
FETCH-STAFF     82
Name: count, dtype: int64

In [865]:
usersData._id.value_counts()

_id
54943462e4b07e684157a532    20
5fc961c3b8cfca11a077dd33    20
5ff5d15aeb7c7d12096d91a2    18
5fa41775898c7a11a6bcef3e    18
59c124bae4b0299e55b0f330    18
                            ..
6004a965e257124ec6b9a39f     1
600746fd6e64691717e8cfb5     1
60074b996e64691717e8f11a     1
60074246325c8a12289e22a0     1
60088e5d633aab121bb8e5cf     1
Name: count, Length: 212, dtype: int64

From above, found that there are duplicates in the User table.

In [866]:
d = {'CONSUMER':'CONSUMER',
     'FETCH-STAFF': 'FETCH_STAFF'} # SQL does not use "-" as enum
usersData.role = usersData.role.apply(lambda x: d[x])

In [867]:
usersData.drop_duplicates(inplace= True) #default use all columns to determine duplicate
# Load data into database table `user`
usersData.to_sql(name = "User", con=engine, if_exists='append', index=False)

212

In [874]:
hset = set(usersData._id)

In [877]:
unknownUserIds = []
for idx, id in enumerate(receiptsData.userId):
    if id not in hset:
        unknownUserIds.append(id)
print("{} user ids in Receipt table that doesn't have matched user ids in User tables\n".format(len(unknownUserIds)))

148 user ids in Receipt table that doesn't have matched user ids in User tables



In [883]:
# b) Load receipt table into database
print("number of receipt id that has duplicate {}\n".format(sum(receiptsData._id.value_counts() > 1)))
receiptsData.to_sql(name = "Receipt", con=engine, if_exists='append', index=False)

# c) Load Brand table into database
brandsData.to_sql(name = "Brand", con=engine, if_exists='append', index=False)

# d) Load ReceiptRewardItem into database
rewardsReceiptItems.to_sql(name = "ReceiptRewardItem", con=engine, if_exists='append', index=False)

number of receipt id that has duplicate 0



6941

## 2: Write queries that directly answer predetermined questions from a business stakeholder

2a. What are the top 5 brands by receipts scanned for most recent month?

1. The most recent month in the dataset is "2021-03". However, none of the receipts scanned in this month have a brandCode in the receipt record. Unless I missed out that there is another column that could relate to the Brand data in the receipt table, we have to push back months to the most recent month that has receipts of non-missing brandCode to get the top five brands scanned for most recent month.  

2. Use `query1`, it finds the most recent month that has receipts which has valid brandCode, and count the total number of receipts scanned that purchased items from each brand, then get the top 5 brands that has the highest number of receipts scanned.  
The result shows that "2021-02" is the most recent month that has receipts with valid brandCode, and there are in total 3 such brands.

In [1207]:
query_to_df(engine, """
            SELECT DATE_FORMAT(MAX(dateScanned), '%Y-%m') AS most_recent_month 
            FROM Receipt RIGHT JOIN ReceiptRewardItem
                    ON Receipt._id = ReceiptRewardItem.receiptId
            """)

,most_recent_month
0,2021-03


In [1216]:
# What are the top 5 brands by receipts scanned for most recent month?
query1 = """
    WITH MostRecentMonth AS (
    SELECT DATE_FORMAT(MAX(re.dateScanned), '%Y-%m') AS most_recent_month 
    FROM Receipt AS re
    RIGHT JOIN ReceiptRewardItem AS item ON re._id = item.receiptId 
    WHERE item.brandCode <> 'NULL'
    )

    SELECT COUNT(DISTINCT(re._id)) as numReceipts,
            item.brandCode,
            (SELECT most_recent_month FROM MostRecentMonth) AS most_recent_month
    FROM 
    Receipt as re RIGHT JOIN ReceiptRewardItem AS item ON re._id = item.receiptId
    WHERE item.brandCode <> 'NULL' AND 
        DATE_FORMAT(re.dateScanned, '%Y-%m') = 
            (SELECT most_recent_month FROM MostRecentMonth)
    GROUP BY item.brandCode
    ORDER BY numReceipts DESC
    LIMIT 5
"""

In [1217]:
df = query_to_df(engine, query1)
# df.sort_values(by= ["dateScanned"], ascending=False)
df

,numReceipts,brandCode,most_recent_month
0,3,BRAND,2021-02
1,2,MISSION,2021-02
2,1,VIVA,2021-02


2b. How does the ranking of the top 5 brands by receipts scanned for the recent month compare to the ranking for the previous month?

Adopt a similar strategy, in the query below, it first find the second most recent month such that has receipts with branCode. Then use it to filter out all receipts scanned in that month.
The results show that the five brands that has the most receipts scanned are:  
'BEN AND JERRYS', 'PEPSI', 'FOLGERS', 'KRAFT', "KELLOGG'S"

Compared the the result from executing query1, "BRAND", "MISSION", "VIVA", they are no overlapps. 
The 5 brands has the most recetips scanned in the previous recent month are not in the list of the top 5 in the most recent month at all.

In [1220]:
query2 = """
    WITH SecondMostRecentMonth AS (
        SELECT DISTINCT DATE_FORMAT(dateScanned, '%Y-%m') AS month 
            FROM Receipt RIGHT JOIN ReceiptRewardItem
                ON Receipt._id = ReceiptRewardItem.receiptId 
            WHERE ReceiptRewardItem.brandCode <> 'NULL'
        ORDER BY DATE_FORMAT(dateScanned, '%Y-%m') DESC 
        LIMIT 1 OFFSET 1
    )

    SELECT COUNT(DISTINCT(re._id)) as numReceipts,
            item.brandCode as Top5,
            (SELECT month FROM SecondMostRecentMonth) AS second_most_recent_month
    FROM 
    Receipt as re RIGHT JOIN ReceiptRewardItem AS item ON re._id = item.receiptId
    WHERE item.brandCode <> 'NULL' AND 
            DATE_FORMAT(re.dateScanned, '%Y-%m') = (SELECT month FROM SecondMostRecentMonth)
    GROUP BY item.brandCode
    ORDER BY numReceipts DESC
    LIMIT 5
"""
res2 = query_to_df(engine, query2)
res2

,numReceipts,Top5,second_most_recent_month
0,32,BEN AND JERRYS,2021-01
1,23,PEPSI,2021-01
2,23,FOLGERS,2021-01
3,22,KRAFT,2021-01
4,22,KELLOGG'S,2021-01


2c. When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

Findings:   
1. According to the values of rewardsReceiptStatus’:  
    "FINISHED",  
    "SUBMITTED",  
    "REJECTED",   
    "PENDING",   
    "FLAGGED",   
I can safely assume that "FINISHED" is equivalent to status "Accepted" 
2. According to the result of the following query, receipts with "Accecpted" reward status has higher average spend

In [1203]:
query3 = """
    SELECT AVG(totalSpent), 
        CASE 
            WHEN rewardsReceiptStatus = 'FINISHED' THEN 'ACCEPTED'
            ELSE rewardsReceiptStatus 
        END AS rewardsReceiptStatus 
    FROM Receipt
    WHERE totalSpent <> 'NULL'
    GROUP BY rewardsReceiptStatus
    HAVING rewardsReceiptStatus = 'FINISHED' OR rewardsReceiptStatus = 'REJECTED'
        """
query_to_df(conn=engine, query = query3)

,AVG(totalSpent),rewardsReceiptStatus
0,24.355146,REJECTED
1,81.167694,ACCEPTED


2d. When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?



Findings: 
1. 1. According to the values of rewardsReceiptStatus’:  
    "FINISHED",  
    "SUBMITTED",  
    "REJECTED",   
    "PENDING",  
    "FLAGGED",   
I can safely assume that "FINISHED" is equivalent to status "Accepted"   
2. According to the result of the following query, receipts with "Accecpted" reward status has higher total number of items purchaed

In [1205]:
query4 = """
    SELECT SUM(purchasedItemCount) as Total, 
            CASE 
                WHEN rewardsReceiptStatus = 'FINISHED' THEN 'ACCEPTED'
                ELSE rewardsReceiptStatus 
            END AS rewardsReceiptStatus  
    FROM Receipt
    WHERE purchasedItemCount <> 'NULL'
    GROUP BY rewardsReceiptStatus
    HAVING rewardsReceiptStatus = 'FINISHED' OR rewardsReceiptStatus = 'REJECTED'
    """
query_to_df(conn=engine, query = query4)

,Total,rewardsReceiptStatus
0,173.0,REJECTED
1,8184.0,ACCEPTED


2e. Which brand has the most spend among users who were created within the past 6 months?

Steps explaned:  
i. How do I define "past 6 month": assuming the most recent month in the dataset is the most recent moneth, which is "2021-03". 
The most recent month by dateScanned in receipt table is "2021-03", them most recent month by createdDate in user table is "2021-02", so I pick the maximum among the two as the most recent month.

Then I trace backward 5 months and include the most recent month, therefore the past 6 months are from "2020-10" to "2021-03".  

ii. Filter out all receipts that were scanned by users created within the past 6 months 
 
iii. Join these receipts with items, group by brand, and sum up the total spent


Finding(s): 'BEN AND JERRYS' is the brand has the most total spend among users who were created within the past 6 months.


In [1301]:
query5 = """
    WITH maxMonth AS (
        SELECT MAX(month) AS max_month
        FROM (
            (SELECT dateScanned AS month
            FROM receipt
            ORDER BY dateScanned DESC
            LIMIT 1)
            UNION ALL
            (SELECT createdDate AS month
            FROM user
            ORDER BY createdDate DESC
            LIMIT 1)
            ) AS combined_results
        ),
    receiptsOfRecentUsers AS (
        SELECT r._id, r.totalSpent, u.createdDate
        FROM receipt AS r
        LEFT JOIN user AS u ON r.userId = u._id
        WHERE DATE_FORMAT(u.createdDate, '%Y-%m') >= (SELECT DATE_FORMAT(DATE_SUB(max_month, INTERVAL 5 MONTH), '%Y-%m') FROM maxMonth)
    )
    SELECT item.brandCode, ROUND(SUM(roru.totalSpent), 2) AS Total
    FROM receiptsOfRecentUsers AS roru
    LEFT JOIN ReceiptRewardItem AS item ON roru._id = item.receiptId
    WHERE item.brandCode <> 'NULL'
    GROUP BY item.brandCode
    ORDER BY Total DESC
    LIMIT 1;
    """
query_to_df(conn=engine, query = query5)

,brandCode,Total
0,BEN AND JERRYS,197337.68


2f. Which brand has the most transactions among users who were created within the past 6 months?

How do I define "past 6 month": starting for the most recent month in the dataset is the most recent moneth, then trace backward 5 months and include the most recent month.

Findings: "HY-VEE" is the brand with the most transactions among users who were created within the past 6 months.

In [1304]:
query6 = """
    WITH maxMonth AS (
        SELECT MAX(month) AS max_month
        FROM (
            (SELECT dateScanned AS month
            FROM receipt
            ORDER BY dateScanned DESC
            LIMIT 1)
            UNION ALL
            (SELECT createdDate AS month
            FROM user
            ORDER BY createdDate DESC
            LIMIT 1)
            ) AS combined_results
        ), 
        receiptsOfRecentUsers AS (
        Select r._id, r.totalSpent, u.createdDate
        FROM receipt AS r LEFT JOIN user as u ON r.userId = u._id
        WHERE DATE_FORMAT(u.createdDate, '%Y-%m') >= (SELECT DATE_FORMAT(DATE_SUB(max_month, INTERVAL 5 MONTH), '%Y-%m') FROM maxMonth)
        )

    SELECT item.brandCode, COUNT(*) as Total
        FROM receiptsOfRecentUsers AS r LEFT JOIN ReceiptRewardItem AS item
            ON r._id = item.receiptId
        WHERE item.brandCode <> 'NULL'
        GROUP BY item.brandCode
        ORDER BY Total DESC
        LIMIT 1
    """
query_to_df(conn=engine, query = query6)

,brandCode,Total
0,HY-VEE,291


In [946]:
sum(rewardsReceiptItems.barcode.isna())

3851

In [959]:
rewardsReceiptItems[~rewardsReceiptItems.brandCode.isna()][["brandCode"]]

,brandCode
8,MISSION
9,BRAND
150,KRAFT EASY CHEESE
306,PEPSI
308,DORITOS
...,...
6752,BRAND
6754,MISSION
6824,VIVA
6846,BRAND


In [889]:
rewardsReceiptItems.columns

Index(['barcode', 'description', 'finalPrice', 'itemPrice', 'needsFetchReview',
       'partnerItemId', 'preventTargetGapPoints', 'quantityPurchased',
       'userFlaggedBarcode', 'userFlaggedNewItem', 'userFlaggedPrice',
       'userFlaggedQuantity', 'receiptId', 'needsFetchReviewReason',
       'pointsNotAwardedReason', 'pointsPayerId', 'rewardsGroup',
       'rewardsProductPartnerId', 'userFlaggedDescription',
       'originalMetaBriteBarcode', 'originalMetaBriteDescription', 'brandCode',
       'competitorRewardsGroup', 'discountedItemPrice',
       'originalReceiptItemText', 'itemNumber',
       'originalMetaBriteQuantityPurchased', 'pointsEarned', 'targetPrice',
       'competitiveProduct', 'originalFinalPrice',
       'originalMetaBriteItemPrice', 'deleted', 'priceAfterCoupon',
       'metabriteCampaignId'],
      dtype='object')

In [894]:
rewardsReceiptItems.shape[0]

6941

In [908]:
sum(rewardsReceiptItems.quantityPurchased.isna())

174

In [945]:
sum(rewardsReceiptItems.brandCode.isna())

4341

## 3. Data Quality Issues Evaluation

a. There are Users who submitted receipts but do not have records in the User table  
b. Duplicate Users exist in the User table  
c. The are a lot of receipt reward items that don't have a brandCode or a field that could be mapped to an existing Brand record
d. There's no id assigned for each reward items  
e. Overlapping Brand categories

In [884]:
# brandsData.categoryCode.value_counts()
# brandsData.category.value_counts()
# sum(brandsData["_id.$oid"].value_counts() != 1)

In [602]:
sum(brandsData.groupby(['categoryCode', 'brandCode'], as_index = False).count()["barcode"] != 1)
# In addition to each brand id, this compound key (categoryCode, brandCode) is also unique

0

## 4. Communicate with Stakeholders

What questions do you have about the data?  
How did you discover the data quality issues?  
What do you need to know to resolve the data quality issues?  
What other information would you need to help you optimize the data assets you're trying to create?  
What performance and scaling concerns do you anticipate in production and how do you plan to address them?  

#### DRAFT AREA BELOW

In [1045]:
raw_rec = dataDict['receipts']
raw_rec[raw_rec["_id.$oid"] == "602538b30a7214d8e9000248"]

,bonusPointsEarned,bonusPointsEarnedReason,pointsEarned,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId,_id.$oid,createDate.$date,dateScanned.$date,finishedDate.$date,modifyDate.$date,pointsAwardedDate.$date,purchaseDate.$date
951,25.0,COMPLETE_NONPARTNER_RECEIPT,25.0,1.0,"[{'barcode': '042800108005', 'brandCode': 'BRA...",FINISHED,10.00,60253861efa6017a44dc6b50,602538b30a7214d8e9000248,1613052083000,1613052083000,1.613052e+12,1613052084000,1.613052e+12,1.612397e+12


In [ ]:
raw_rec = dataDict['receipts']
raw_rec[raw_rec["_id.$oid"] == "602538b30a7214d8e9000248"]

,bonusPointsEarned,bonusPointsEarnedReason,pointsEarned,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId,_id.$oid,createDate.$date,dateScanned.$date,finishedDate.$date,modifyDate.$date,pointsAwardedDate.$date,purchaseDate.$date
951,25.0,COMPLETE_NONPARTNER_RECEIPT,25.0,1.0,"[{'barcode': '042800108005', 'brandCode': 'BRA...",FINISHED,10.00,60253861efa6017a44dc6b50,602538b30a7214d8e9000248,1613052083000,1613052083000,1.613052e+12,1613052084000,1.613052e+12,1.612397e+12


In [1070]:
indices = []
for idx, it in enumerate(raw_rec.rewardsReceiptItemList):
    try:
        if type(it) is not list and pd.isna(it):
            continue
    except ValueError:
        print(it)
        print(len(it))
        # break
    
    for item in it:
        if "brandCode" in item:
            print(idx, item)
            indices.append(idx)
            break
        
        

6 {'brandCode': 'MISSION', 'competitorRewardsGroup': 'TACO BELL TACO SHELLS', 'description': 'MSSN TORTLLA', 'discountedItemPrice': '2.23', 'finalPrice': '2.23', 'itemPrice': '2.23', 'originalReceiptItemText': 'MSSN TORTLLA', 'partnerItemId': '1009', 'quantityPurchased': 1}
7 {'barcode': '046000832517', 'brandCode': 'BRAND', 'description': 'Old El Paso Mild Chopped Green Chiles, 4.5 Oz', 'finalPrice': '10.00', 'itemPrice': '10.00', 'partnerItemId': '0', 'pointsNotAwardedReason': 'Action not allowed for user and CPG', 'pointsPayerId': '5332f5f3e4b03c9a25efd0ae', 'quantityPurchased': 1, 'rewardsGroup': 'OLD EL PASO BEANS & PEPPERS', 'rewardsProductPartnerId': '5332f5f3e4b03c9a25efd0ae'}
47 {'barcode': '044000000745', 'brandCode': 'KRAFT EASY CHEESE', 'competitiveProduct': True, 'competitorRewardsGroup': 'SARGENTO RICOTTA CHEESE', 'description': '-Cheddar', 'discountedItemPrice': '1.00', 'finalPrice': '1.00', 'itemNumber': '044000000745', 'itemPrice': '1.00', 'partnerItemId': '1030', 'qua

In [1074]:
recs_with_barcode = raw_rec.iloc[indices]

# receiptsDataRenameCols = {col: cleanColName(col) for col in receiptsData.columns if "." in col}
recs_with_barcode = recs_with_barcode.rename(columns=receiptsDataRenameCols)

In [1075]:
for col in ['createDate', 'dateScanned',
        'finishedDate', 'modifyDate', 
        'pointsAwardedDate', 'purchaseDate']:
    recs_with_barcode[col] = pd.to_datetime(recs_with_barcode[col], unit="ms")
recs_with_barcode

,bonusPointsEarned,bonusPointsEarnedReason,pointsEarned,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId,_id,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,purchaseDate
6,5.0,All-receipts receipt bonus,5.0,1.0,"[{'brandCode': 'MISSION', 'competitorRewardsGr...",FINISHED,2.23,5ff1e194b6a9d73a3a9f1052,5ff1e1cd0a720f052300056f,2021-01-03 15:25:01,2021-01-03 15:25:01,2021-01-03 15:25:02,2021-01-03 15:25:02,2021-01-03 15:25:02,2021-01-03 15:25:01
7,500.0,"Receipt number 2 completed, bonus point schedu...",500.0,1.0,"[{'barcode': '046000832517', 'brandCode': 'BRA...",FINISHED,10.00,5ff1e194b6a9d73a3a9f1052,5ff1e1a40a720f0523000569,2021-01-03 15:24:20,2021-01-03 15:24:20,2021-01-03 15:24:21,2021-01-03 15:24:21,2021-01-03 15:24:21,2020-12-27 00:00:00
47,25.0,COMPLETE_NONPARTNER_RECEIPT,25.0,1.0,"[{'barcode': '044000000745', 'brandCode': 'KRA...",FINISHED,1.00,5964eb07e4b03efd0c0f267b,5ff29be20a7214ada1000571,2021-01-04 04:38:58,2021-01-04 04:38:58,2021-01-04 04:39:06,2021-01-04 04:39:09,2021-01-04 04:39:06,2021-01-03 00:00:00
108,750.0,"Receipt number 1 completed, bonus point schedu...",793.1,16.0,[{'description': 'EMIL' S SAUSAGE MUSHROOM PIZ...,FINISHED,52.12,5ff616a68f142f11dd189163,5ff618e30a7214ada10005fa,2021-01-06 20:09:07,2021-01-06 20:09:07,2021-01-06 20:09:42,2021-01-06 20:09:45,2021-01-06 20:09:42,2021-01-06 00:00:00
123,NaN,NaN,0.0,2.0,"[{'brandCode': 'WINGSTOP', 'description': '12 ...",FINISHED,6.18,5ff7741e04929111f6e90902,5ff774200a7214ada1000638,2021-01-07 20:50:40,2021-01-07 20:50:40,2021-01-07 20:50:40,2021-01-07 20:50:40,NaT,2021-01-07 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688,25.0,COMPLETE_NONPARTNER_RECEIPT,25.0,1.0,"[{'barcode': '070470149394', 'brandCode': 'BRA...",FINISHED,10.00,60189c74c8b50e11d8454eff,60189cc10a7214ad28000050,2021-02-02 00:28:49,2021-02-02 00:28:49,2021-02-02 00:28:50,2021-02-02 00:28:50,2021-02-02 00:28:50,2021-01-26 00:00:00
691,5.0,All-receipts receipt bonus,5.0,1.0,"[{'brandCode': 'MISSION', 'competitorRewardsGr...",FINISHED,2.23,60189c74c8b50e11d8454eff,60189cae0a7214ad2800004f,2021-02-02 00:28:30,2021-02-02 00:28:30,2021-02-02 00:28:31,2021-02-02 00:28:31,2021-02-02 00:28:31,2021-02-02 00:28:30
917,750.0,"Receipt number 1 completed, bonus point schedu...",789.2,2.0,"[{'description': 'BROWNIE BRITLE', 'discounted...",FINISHED,5.88,60245433efa60112282c1859,602454340a720f05a80001ae,2021-02-10 21:46:28,2021-02-10 21:46:28,2021-02-10 21:46:29,2021-02-10 21:46:37,2021-02-10 21:46:29,2021-02-01 00:00:00
951,25.0,COMPLETE_NONPARTNER_RECEIPT,25.0,1.0,"[{'barcode': '042800108005', 'brandCode': 'BRA...",FINISHED,10.00,60253861efa6017a44dc6b50,602538b30a7214d8e9000248,2021-02-11 14:01:23,2021-02-11 14:01:23,2021-02-11 14:01:24,2021-02-11 14:01:24,2021-02-11 14:01:24,2021-02-04 00:00:00


In [1077]:
recs_with_barcode.sort_values(by = ['dateScanned'], ascending= False)

,bonusPointsEarned,bonusPointsEarnedReason,pointsEarned,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId,_id,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,purchaseDate
951,25.0,COMPLETE_NONPARTNER_RECEIPT,25.0,1.0,"[{'barcode': '042800108005', 'brandCode': 'BRA...",FINISHED,10.00,60253861efa6017a44dc6b50,602538b30a7214d8e9000248,2021-02-11 14:01:23,2021-02-11 14:01:23,2021-02-11 14:01:24,2021-02-11 14:01:24,2021-02-11 14:01:24,2021-02-04 00:00:00
955,5.0,All-receipts receipt bonus,5.0,1.0,"[{'brandCode': 'MISSION', 'competitorRewardsGr...",FINISHED,2.23,60253861efa6017a44dc6b50,602538740a7214d8e9000246,2021-02-11 14:00:20,2021-02-11 14:00:20,2021-02-11 14:00:22,2021-02-11 14:00:23,2021-02-11 14:00:22,2021-02-11 14:00:20
917,750.0,"Receipt number 1 completed, bonus point schedu...",789.2,2.0,"[{'description': 'BROWNIE BRITLE', 'discounted...",FINISHED,5.88,60245433efa60112282c1859,602454340a720f05a80001ae,2021-02-10 21:46:28,2021-02-10 21:46:28,2021-02-10 21:46:29,2021-02-10 21:46:37,2021-02-10 21:46:29,2021-02-01 00:00:00
688,25.0,COMPLETE_NONPARTNER_RECEIPT,25.0,1.0,"[{'barcode': '070470149394', 'brandCode': 'BRA...",FINISHED,10.00,60189c74c8b50e11d8454eff,60189cc10a7214ad28000050,2021-02-02 00:28:49,2021-02-02 00:28:49,2021-02-02 00:28:50,2021-02-02 00:28:50,2021-02-02 00:28:50,2021-01-26 00:00:00
691,5.0,All-receipts receipt bonus,5.0,1.0,"[{'brandCode': 'MISSION', 'competitorRewardsGr...",FINISHED,2.23,60189c74c8b50e11d8454eff,60189cae0a7214ad2800004f,2021-02-02 00:28:30,2021-02-02 00:28:30,2021-02-02 00:28:31,2021-02-02 00:28:31,2021-02-02 00:28:31,2021-02-02 00:28:30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,25.0,COMPLETE_NONPARTNER_RECEIPT,25.0,1.0,"[{'barcode': '021908236865', 'brandCode': 'BRA...",FINISHED,10.00,5ff73b90eb7c7d31ca8a452b,5ff73bdb0a720f0523000605,2021-01-07 16:50:35,2021-01-07 16:50:35,2021-01-07 16:50:35,2021-01-07 16:50:35,2021-01-07 16:50:35,2020-12-31 00:00:00
108,750.0,"Receipt number 1 completed, bonus point schedu...",793.1,16.0,[{'description': 'EMIL' S SAUSAGE MUSHROOM PIZ...,FINISHED,52.12,5ff616a68f142f11dd189163,5ff618e30a7214ada10005fa,2021-01-06 20:09:07,2021-01-06 20:09:07,2021-01-06 20:09:42,2021-01-06 20:09:45,2021-01-06 20:09:42,2021-01-06 00:00:00
47,25.0,COMPLETE_NONPARTNER_RECEIPT,25.0,1.0,"[{'barcode': '044000000745', 'brandCode': 'KRA...",FINISHED,1.00,5964eb07e4b03efd0c0f267b,5ff29be20a7214ada1000571,2021-01-04 04:38:58,2021-01-04 04:38:58,2021-01-04 04:39:06,2021-01-04 04:39:09,2021-01-04 04:39:06,2021-01-03 00:00:00
6,5.0,All-receipts receipt bonus,5.0,1.0,"[{'brandCode': 'MISSION', 'competitorRewardsGr...",FINISHED,2.23,5ff1e194b6a9d73a3a9f1052,5ff1e1cd0a720f052300056f,2021-01-03 15:25:01,2021-01-03 15:25:01,2021-01-03 15:25:02,2021-01-03 15:25:02,2021-01-03 15:25:02,2021-01-03 15:25:01


In [1156]:
k = recs_with_barcode.explode("rewardsReceiptItemList")["rewardsReceiptItemList"]
k

6      {'brandCode': 'MISSION', 'competitorRewardsGro...
7      {'barcode': '046000832517', 'brandCode': 'BRAN...
47     {'barcode': '044000000745', 'brandCode': 'KRAF...
108    {'description': 'EMIL' S SAUSAGE MUSHROOM PIZZ...
108    {'description': 'KLARBRUNN 12PK 12 FL OZ', 'di...
                             ...                        
691    {'brandCode': 'MISSION', 'competitorRewardsGro...
917    {'description': 'BROWNIE BRITLE', 'discountedI...
917    {'barcode': '036000494129', 'brandCode': 'VIVA...
951    {'barcode': '042800108005', 'brandCode': 'BRAN...
955    {'brandCode': 'MISSION', 'competitorRewardsGro...
Name: rewardsReceiptItemList, Length: 5353, dtype: object

In [1158]:
k.iloc[6]

{'description': 'KLARBRUNN 12PK 12 FL OZ',
 'discountedItemPrice': '7.53',
 'finalPrice': '7.53',
 'itemPrice': '7.53',
 'originalReceiptItemText': 'KLARBRUNN 12PK 12 FL OZ',
 'partnerItemId': '1019',
 'quantityPurchased': 3}

In [1159]:
k.iloc[691]

{'description': 'MONTEREY SLCD BABY B',
 'discountedItemPrice': '2.50',
 'finalPrice': '2.50',
 'itemPrice': '2.50',
 'originalReceiptItemText': 'MONTEREY SLCD BABY B',
 'partnerItemId': '1171',
 'quantityPurchased': 1}

In [1185]:
g0 = rewardsReceiptItems.iloc[list(rewardsReceiptItems.groupby(["partnerItemId","barcode"]).groups[('0', 'B076FJ92M4')])]
g0

,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,userFlaggedNewItem,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
6858,B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,NaN,0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN
6860,B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,NaN,0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN
6865,B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,NaN,0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN
6867,B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,NaN,0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN
6869,B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,NaN,0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN
6871,B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,NaN,0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN
6873,B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,NaN,0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN
6875,B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,NaN,0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN
6877,B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,NaN,0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN
6879,B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,NaN,0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN


In [1184]:
rewardsReceiptItems.groupby(["partnerItemId","barcode"]).groups

{('0', '001111132666'): [437], ('0', '001111147332'): [467], ('0', '007940018942'): [6675], ('0', '016000128620'): [6718], ('0', '016000147096'): [6079], ('0', '016000151024'): [2623], ('0', '016000156234'): [391, 413], ('0', '016000898264'): [6671], ('0', '021908236865'): [335], ('0', '036000320893'): [6092, 6096, 6100, 6104, 6108, 6117, 6121, 6125, 6129, 6133, 6137, 6143, 6148, 6152, 6156, 6160, 6164, 6168, 6633, 6638, 6642, 6666, 6700, 6703, 6717, 6719], ('0', '042800108005'): [6846], ('0', '046000832517'): [9], ('0', '070470149394'): [6752], ('0', '070470400235'): [6115, 6116], ('0', '072785048119'): [555], ('0', '079400245328'): [453, 510], ('0', '089203700016'): [447], ('0', '2700710686201'): [6056, 6057], ('0', '305210154278'): [432], ('0', '665290001184'): [6864], ('0', '822142991523'): [677], ('0', 'B076FJ92M4'): [6858, 6860, 6865, 6867, 6869, 6871, 6873, 6875, 6877, 6879, 6881, 6883, 6885, 6887, 6889, 6891, 6893, 6895, 6897, 6899, 6901, 6903, 6905, 6907, 6909, 6911, 6913, 691

In [1192]:
rewardsReceiptItems.shape[0]

6941

In [1193]:
rewardsReceiptItems.drop_duplicates(['barcode','partnerItemId', 'description']).shape[0]

5814

In [1199]:
rewardsReceiptItems.describe()

,quantityPurchased,userFlaggedQuantity,originalMetaBriteQuantityPurchased
count,6767.000000,299.000000,15.000000
mean,1.386139,1.872910,1.200000
std,1.204363,1.314823,0.414039
min,1.000000,1.000000,1.000000
25%,1.000000,1.000000,1.000000
50%,1.000000,1.000000,1.000000
75%,1.000000,3.000000,1.000000
max,17.000000,5.000000,2.000000
